In [1]:
import pandas as pd
import numpy as np

In [2]:
def load_data():
    df_prices = pd.read_csv('prices-split-adjusted.csv')
    df_prices = df_prices[['date', 'symbol', 'close']]
    #df_prices['date'] = pd.to_datetime(df_prices['date'])
    #df_prices['delta'] = df_prices['close'].diff()
    #df_prices = df_prices.iloc[1:, [0, 1, 3]]

    df_securities = pd.read_csv('securities.csv')
    df_securities = df_securities[['Ticker symbol', 'Security']]
    df_securities.columns = ['ticker', 'name']

    return df_prices, df_securities

def max_difference(A, low=0, high=None):
    if high is None:
        high = len(A) - 1

    # Base case: if the array has only one element, return it as both min and max
    if low == high:
        return (A[low], low), (A[low], low), 0  # (min_value, min_index), (max_value, max_index), max_diff

    # Divide
    mid = (low + high) // 2

    # Conquer
    minLeft, maxLeft, maxDiffLeft = max_difference(A, low, mid)
    minRight, maxRight, maxDiffRight = max_difference(A, mid + 1, high)

    # Combine
    maxDiffAcross = maxRight[0] - minLeft[0] if maxRight[1] > minLeft[1] else 0

    # Find overall max difference
    maxDiff = max(maxDiffLeft, maxDiffRight, maxDiffAcross)

    # Determine overall min and max values and their indices
    # Lambda function is to make sure that it comparews the actual value in the tuple
    overall_min = min(minLeft, minRight, key=lambda x: x[0])
    overall_max = max(maxLeft, maxRight, key=lambda x: x[0])

    return overall_min, overall_max, maxDiff

df_prices, df_securities = load_data()

ticker = df_securities.iloc[3]['ticker']

df = df_prices.loc[df_prices['symbol'] == ticker, ['date', 'close']]

minTuple, maxTuple, diff = max_difference(np.array(df['close']))
minPrice, minIndex = minTuple
maxPrice, maxIndex = maxTuple
minDate = df['date'].iloc[minIndex]
maxDate = df['date'].iloc[maxIndex]
security = df_securities.loc[df_securities['ticker'] == ticker, 'name'].tolist()[0]

print(f'Best stock to buy: {security} on {minDate} and sell on {maxDate} with profit of ${diff:.2f}')

Best stock to buy: Accenture plc on 2010-06-08 and sell on 2016-12-13 with profit of $88.96


In [11]:
df_securities.shape[0]

505

In [6]:
# def max_difference_iterative(A):
#     if len(A) == 0:
#         return None, None, 0  # In case of empty array

#     min_value = A[0]
#     min_index = 0
#     max_value = A[0]
#     max_index = 0
#     max_diff = 0

#     for i in range(1, len(A)):
#         if A[i] < min_value:
#             min_value = A[i]
#             min_index = i
#         elif A[i] - min_value > max_diff:
#             max_diff = A[i] - min_value
#             max_value = A[i]
#             max_index = i

#     return (min_value, min_index), (max_value, max_index), max_diff

# df_prices, df_securities = load_data()

# for i in range(df_securities.shape[0]):
#     ticker = df_securities.iloc[i]['ticker']
#     df = df_prices.loc[df_prices['symbol'] == ticker, ['date', 'close']]

#     minTuple, maxTuple, diff = max_difference_iterative(np.array(df['close']))
#     if minTuple is not None and maxTuple is not None:
#         minPrice, minIndex = minTuple
#         maxPrice, maxIndex = maxTuple
#         minDate = df['date'].iloc[minIndex]
#         maxDate = df['date'].iloc[maxIndex]
#         security = df_securities.loc[df_securities['ticker'] == ticker, 'name'].tolist()[0]

#         print(f'Best stock to buy: {security} on {minDate} and sell on {maxDate} with profit of ${diff:.2f}')


In [37]:
def max_difference(A, low=0, high=None):
    if high is None:
        high = len(A) - 1

    # Base case: if the array has only one element, return it as both min and max
    if low == high:
        return A[low], A[low], 0  # min_value, max_value, max_diff

    # Base case: if the array has two elements, handle separately
    if high == low + 1:
        min_val = min(A[low], A[high])
        max_val = max(A[low], A[high])
        return min_val, max_val, max_val - min_val

    # Divide
    mid = (low + high) // 2

    # Conquer
    minLeft, maxLeft, maxDiffLeft = max_difference(A, low, mid)
    minRight, maxRight, maxDiffRight = max_difference(A, mid + 1, high)

    # Combine
    maxDiffAcross = maxRight - minLeft

    # Find overall max difference
    maxDiff = max(maxDiffLeft, maxDiffRight, maxDiffAcross)

    # Determine overall min and max values
    overall_min = min(minLeft, minRight)
    overall_max = max(maxLeft, maxRight)

    return overall_min, overall_max, maxDiff

df_prices, df_securities = load_data()

max_diff = 0
most_min = 0
most_max = 0
best_ticker = None
count = 0
for i in range(df_securities.shape[0]):

    ticker = df_securities.iloc[i]['ticker']
    
    df = df_prices.loc[df_prices['symbol'] == ticker, ['date', 'close']]

    if df.shape[0] != 0:
        minimum, maximum, diff = max_difference(np.array(df['close']))
        count += 1

        if diff > max_diff:
            max_diff = diff
            most_min = minimum
            most_max = maximum
            best_ticker = ticker
        else:
            pass
    else:
        pass
 
minDate = df_prices.loc[(df_prices['close'] == most_min) & (df_prices['symbol'] == best_ticker), ['date']].values[0][0]
maxDate = df_prices.loc[(df_prices['close'] == most_max) & (df_prices['symbol'] == best_ticker), ['date']].values[0][0]
security = df_securities.loc[df_securities['ticker'] == best_ticker, 'name'].tolist()[0]

print(f'Best stock to buy: {security} on {minDate} and sell on {maxDate} with profit of ${max_diff:.2f}')
print(count)


Best stock to buy: Priceline.com Inc on 2010-06-09 and sell on 2016-11-08 with profit of $1402.94
501
